# 예시문제

In [ ]:
# 예시문제 링크
mtcars_url = "https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv"

## 작업형 1유형
mcars 데이터셋의 qsec 컬럼을 최소-최대 척도(Min-Max Scale)로 변환한 후 0.5보다 큰 값을 갖는 레코드 수 구하기


In [ ]:
import pandas as pd
df = pd.read_csv(mtcars_url)
# print(df.head())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['qsec'] = scaler.fit_transform(df[['qsec']])
# print(df.head())
# print(help(MinMaxScaler))

cond = df['qsec'] > 0.5
print(len(df[cond]))

9


## 작업형 2유형
백화점 구매 고객의 성별을 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(test)에 적용해 얻은 성별 예측 결과를 [제출 형식]에 따라 csv 파일로 생성하여 제출
> 예측 결과는 ROC-AUC 평가 지표에 따라 평가

1. 제출 파일명: result_1.csv
2. 예측 성별 컬럼명: pred
3. 제출 컬럼 개수: pred 컬럼 1개
4. 평가용 데이터 개수와 예측 결과 데이터 개수 일치: 2,482개

In [ ]:
customer_train_url = "https://github.com/jjjjunn/Bigdata_analystic/raw/refs/heads/main/%5BDataset%5D%20%EC%9E%91%EC%97%85%ED%98%95%20%EC%A0%9C2%EC%9C%A0%ED%98%95(%EC%98%88%EC%8B%9C)/customer_train.csv"
customer_test_url = "https://github.com/jjjjunn/Bigdata_analystic/raw/refs/heads/main/%5BDataset%5D%20%EC%9E%91%EC%97%85%ED%98%95%20%EC%A0%9C2%EC%9C%A0%ED%98%95(%EC%98%88%EC%8B%9C)/customer_test.csv"
target_url = "https://github.com/jjjjunn/Bigdata_analystic/raw/refs/heads/main/%5BDataset%5D%20%EC%9E%91%EC%97%85%ED%98%95%20%EC%A0%9C2%EC%9C%A0%ED%98%95(%EC%98%88%EC%8B%9C)/y_train.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(customer_train_url, encoding='cp949')
test = pd.read_csv(customer_test_url, encoding='cp949')
# print(train.head())
# print()
# print(test.head())

# 데이터 크기 확인
# print(train.shape, test.shape) # (3500, 10) (2482, 10)

# 데이터 정보 확인
# print(train.info()) # 환불 금액 결측치
# print(test.info()) # 환불 금액 결측치

# 결측치 확인
# print(train.isnull().sum()) # 환불 금액 2295
# print(test.isnull().sum()) # 환불 금액 1611

 # target 변수
target = pd.read_csv(target_url)
target = target['gender']
# print(target.head())
# print(target.shape) # 3500, 2 # target 컬럼: 'gender'

# 범주형 변수 추출
cols = train.select_dtypes(include='O').columns
# print(cols) # 주구매상품, 주구매지점

# 범주형 변수 빈도 확인 (unique)
# print(train[cols].describe(include='O')) # 주구매상품: 42, 주구매지점: 24

# 범주형 변수 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

combine = pd.concat([train, test])
# print(combine.shape) # (5982, 10)

# 결측치 처리
combine['환불금액'] = combine['환불금액'].fillna(0)
# print(combine.isnull().sum())

le = LabelEncoder()
for col in cols:
  combine[col] = le.fit_transform(combine[col])
  # print(f"{col} 인코딩 완료")

n_train = len(train)
train = combine[:n_train]
test = combine[n_train:]

# print(train.shape, test.shape)
# print(train.head())

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train,
    target,
    test_size = 0.2,
    random_state=42)

# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (2800, 10) (700, 10) (2800, 2) (700, 2)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred_proba = rf.predict_proba(X_val)[:, 1]
print(pred_proba[:5])

# 모델 평가
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_val, pred_proba)
print(f"ROC-AUC SCORE: {score}") # 0.6512769042043046

# 최종 예측 및 제출
pred = rf.predict(test)
submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_1.csv', index=False)

# 답안 확인
result = pd.read_csv('result_1.csv')
print(result)

[0.55 0.78 0.35 0.25 0.34]
ROC-AUC SCORE: 0.6512769042043046
      pred
0        0
1        0
2        0
3        1
4        0
...    ...
2477     1
2478     0
2479     1
2480     1
2481     0

[2482 rows x 1 columns]


## 작업형 3유형
타이타닉호에서 생존한 승객 및 사망한 승객의 정보를 포함한 데이터를 이용해 생존 여부를 예측.
1. Gender와 Survive 변수 간의 독립성 검정을 실시했을 때 카이제곱 통계량은?
(반올림하여 소수 셋째 자리까지 게산)
2. Gender, SibSp, Parch, Fare를 독립변수로 사용해 로지스틱 회귀 모델을 실시했을 때 Parch 변수의 계수 값은?
(반올림하여 소수 셋째 자리까지 계산)
3. 2번 문제에서 추정된 로지스틱 회귀 모델에서 SibSp 변수가 한 단위 증가할 대 생존할 오즈비(Odds Ratio) 값은?
(반올림하여 소수 셋째 자리까지 계산)

In [ ]:
titanic_url = "https://raw.githubusercontent.com/jjjjunn/Bigdata_analysis/refs/heads/main/Titanic.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(titanic_url)
print(df.head())

# 1. gender와 Survive 변수 간의 독립성 검정을 실시했을 때 카이제곱 통계량은?
# (반올림하여 소수 셋째 자리까지 게산)
# import scipy.stats
# print(dir(scipy.stats))

from scipy.stats import chi2_contingency

crosstab = pd.crosstab(df['Gender'], df['Survived'])
print(crosstab)
result = chi2_contingency(crosstab)
# print(result)
print(f"카이제곱 통계량: {round(result.statistic, 3)}") # 260.717

# print(help(chi2_contingency)) # chi2_contingency(관측치)

# 2. Gender, SibSp, Parch, Fare를 독립변수로 사용해 로지스틱 회귀 모델을 실시했을 때 Parch 변수의 계수 값은?
# (반올림하여 소수 셋째 자리까지 계산)
from statsmodels.formula.api import logit

model = logit('Survived ~ Gender + SibSp + Parch + Fare', df).fit()
print(model.summary())
# print(model.params['Parch'])
parch = model.params['Parch']
print(f"Parch 변수 계수 값: {round(parch, 3)}") # -0.201

# 3. 2번 문제에서 추정된 로지스틱 회귀 모델에서 SibSp 변수가 한 단위 증가할 대 생존할 오즈비(Odds Ratio) 값은?
# (반올림하여 소수 셋째 자리까지 계산)
import numpy as np

sibsp_odds = np.exp(model.params['SibSp'])
print(f"SibSp 오즈비: {round(sibsp_odds, 3)}") #  0.702

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Gender   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
Su

# 깃허브 url
https://github.com/lovedlim/bigdata_analyst_cert_v2/tree/main

# 제2회 기출

## 작업형 1유형: 문제1
주어진 데이터에서 'views' 컬럼을 기준으로 내림차순 정렬하고, 'views' 컬럼 상위 10개 데이터를 'views' 컬럼 상위 10번째 값으로 대체한 후, 'age' 컬럼이 80 이상인 데이터의 'views' 컬럼 평균 값 구하기
(반올림하여 소수 둘째 자리까지 계산)

In [ ]:
members_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch2/members.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(members_url)
# print(df.head())

# 1-1. 'views' 컬럼 기준으로 내림차순 정렬
df = df.sort_values('views', ascending=False)
# print(df.head())

# 1-2. 상위 10개 데이터를 'views' 컬럼 상위 10번째 값으로 대체
# print(df.head(10)) # 9690.0
view_10 = df['views'].iloc[9]

# print(view_10)
df['views'][:9] = view_10
# print(df.head(11))

# 1-3. 'age' 컬럼이 80 이상인 데이터의 'views' 컬럼 평균 (반올림하여 소수 둘째 자리까지)
cond = df['age'] >= 80
# print(df[cond].head())
cond_df = df[cond]
cond_mean = cond_df['views'].mean()
print(round(cond_mean, 2)) # 5674.04

5674.04


<ipython-input-54-4194197673>:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['views'][:9] = view_10
<ipython-input-54-4194197673>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

## 작업형 1유형: 문제2
주어진 데이터셋에서 앞에서부터 순서대로 80% 데이터만 활용해 'f1' 컬럼 결측치를 중앙 값으로 채우기 전후의 표본 표준편차를 구하고, 두 표본 표준편차의 차이 계산
(두 표본 표준편차 차이는 절댓값으로 계산하고, 반올림하여 소수 둘째 자리까지 계산)

In [ ]:
import pandas as pd

df = pd.read_csv(members_url)
# print(df.head())
# print(df.shape) # (100, 10)

# 2-1. 앞에서부터 순서대로 80% 데이터만 활용
n_df = len(df) * 0.8
n_df = int(n_df)
# print(n_df) # 80
df = df[:n_df]
# print(df.shape) # (80, 10)

# 2-2.  'f1' 컬럼 결측치를 중앙 값으로 채우기 전후의 표본 표준편차 구하기
# 결측치 정보
# print(df['f1'].isnull().sum()) # 25개

# 결측치 채우기 전 표준편차
before_std = df['f1'].std()
# print(before_std) # 20.574853076621935

# 결측치 채우기
f1_median = df['f1'].median()
# print(f1_median) # 68.0
df['f1'] = df['f1'].fillna(f1_median)
after_std = df['f1'].std()
# print(after_std) # 17.010788646613275

# 전후의 표준편차 차이 (절댓값, 반올림하여 소수 둘째 자리까지)
diff = abs(before_std - after_std)
print(round(diff, 2)) # 3.56

3.56


## 작업형 1유형: 문제3
주어진 데이터셋의 'age' 컬럼의 이상치를 모두 더하기.
이상치는 평균으로부터 '표준편차 * 1.5'를 벗어나는 영역을 이상치라고 판단

In [ ]:
df = pd.read_csv(members_url)
# print(df.head())

age_std = df['age'].std()
age_mean = df['age'].mean()
# print(age_std) # 30.442758522588697
# print(age_mean) # 50.963

# 이상치는 평균으로부터 표준편차 * 1.5를 벗어남
out_standard = age_std * 1.5
out_age_over = age_mean + out_standard
out_age_under = age_mean - out_standard

outliers = [out_age_over, out_age_under]
for outlier in outliers:
  print(f"이상치: {outlier}")  #  96.62713778388306 & 5.298862216116952

over_out = df['age'] > out_age_over
under_out = df['age'] < out_age_under

over_cond = df[over_out]
over_sum = (over_cond['age'].sum()) # 4096.6
under_cond = df[under_out]
under_sum = (under_cond['age'].sum()) # 104.5

print(over_sum + under_sum) # 4201.1

이상치: 96.62713778388306
이상치: 5.298862216116952
473.5


## 작업형 2유형
제품 배송 시간에 맞춰 배송되지 않을 확률값 예측

1. 예측할 컬럼: Reached.on.Time_Y.N(0: 정시 도착, 1: 정시 도착하지 않음)
2. 제출할 컬럼: ID, Reached.on.Time_Y.N (정시 도착하지 않을 확률)
3. csv 파일로 제출
4. 모델의 성능은 ROC-AUC 평가지표로 평가

In [ ]:
X_train_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch2/X_train.csv"
X_test_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch2/X_test.csv"
y_train_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch2/y_train.csv"

In [ ]:
import pandas as pd

X_train = pd.read_csv(X_train_url)
X_test = pd.read_csv(X_test_url)
y_train = pd.read_csv(y_train_url)

# print(X_train.shape, X_test.shape, y_train.shape) # (8799, 11) (2200, 11) (8799, 2)

# print(X_train.head())
# print(X_test.head())
# print(y_train.head()) # 타겟 변수

# print(X_train.info()) # 결측치 없음
# print(X_test.info()) # 결측치 없음
# print(y_train.info()) # 결측치 없음

train = X_train
target = y_train
test = X_test

# 범주형 변수 출력
cols = train.select_dtypes(include='O').columns
# print(cols) # 'Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'

# 범주형 변수 unique 값
# print(train.describe(include='O')) # 2에서 5까지
# print(test.describe(include='O')) # 2에서 5까지

# 원-핫 인코딩
# print(help(pd.get_dummies))
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# print(train.shape, test.shape) # (8799, 20) (2200, 20)

# id 컬럼 빼기
train_id = train.pop('ID')
test_id = test.pop('ID')
y_id = target.pop('ID')

# print(train.shape, test.shape, target.shape) # (8799, 19) (2200, 19) (8799, 1)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train,
    target['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=42)

# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (7039, 19) (1760, 19) (7039,) (1760,)
# print(y_val.head())

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred_proba = rf.predict_proba(X_val) # 확률 예측

# 평가
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_val, pred_proba[:,1])
# print(score) # 0.7513118176240968

# 최종 예측 및 제출
pred = rf.predict_proba(test)
submit = pd.DataFrame({'ID': test_id,
                       'Reached.on.Time_Y.N': pred[:, 1]})
submit.to_csv('result_2.csv', index=False)

# 데이터 확인
result = pd.read_csv('result_2.csv')
print(result)

         ID  Reached.on.Time_Y.N
0      8285                 0.27
1     10192                 0.39
2      8675                 0.43
3      5753                 0.37
4       448                 0.88
...     ...                  ...
2195   7193                 0.43
2196   4445                 0.40
2197   7390                 0.37
2198   9977                 0.52
2199   5696                 0.63

[2200 rows x 2 columns]


# 제3회 기출

## 작업형 1유형: 문제1
결측치가 있는 데이터의 행(레코드)을 제거하고, 앞에서부터 70% 데이터만 활용해 'f1' 컬럼 1사분위 값을 구하시오
(소수점 제거 후 정수 출력)

In [ ]:
import pandas as pd

df = pd.read_csv(members_url)
# print(df.head())

# 1-1. 결측치가 있는 데이터 행 제거
# 결측치 확인
# print(df.info())
# print(df.isnull().sum()) # f1, f3, views

# 결측치 제거
df = df.dropna()
# print(df.shape) # (51, 10)

# 1-2. 앞에서부터 70% 데이터만 활용
n_df = len(df)
df_70 = int(n_df * 0.7)
# print(df_70) # 35
df = df[:df_70]
# print(df.shape) # (35, 10)

# 'f1' 컬럼 1사분위 값 구하기 (정수 출력)
f1_q1 = df['f1'].quantile(0.25)
# print(f1_q1) # 57.0
# 정수출력
print(int(f1_q1)) # 57

57


## 작업형 1유형: 문제2
2000년 데이터 중 2000년 평균보다 큰 값의 데이터수 구하기 (index: 연도)

In [ ]:
year_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch3/year.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(year_url, index_col='Unnamed: 0')
# print(df.head())
# print(df.info())

df = df.T
# print(df.head())

# 2-1. 2000년 데이터의 평균
mean = df[2000].mean()
# print(mean) # 100.735

# 2-2. 평균보다 큰 값 데이터 수
cond = df[2000] > mean
print(sum(cond))

100.735
100


## 작업형 1유형: 문제3
결측치가 가장 많은 컬럼명 출력

In [ ]:
import pandas as pd

df = pd.read_csv(members_url)
# print(df.head())

# 결측치가 가장 많은 컬럼
non_values = df.isnull().sum()
# print(non_values) # f1 컬럼 31
print(non_values.idxmax())

f1


## 작업형 2유형
여행 보험 패키지 상품을 구매할 확률값 예측

1. 예측할 컬럼: TravelInsurance(여행 보험 패키지 구매 여부, 0: 구매 안함, 1: 구매)
2. 제출 형태: index, y_pred 컬럼
3. 모델 성능은 ROC-AUC 평가 지표에 따라 평가

In [ ]:
train_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch3/train.csv"
test_url = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch3/test.csv"


In [ ]:
import pandas as pd

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

# print(train.head())
# print(test.head())

# print(train.shape, test.shape) # (1490, 10) (497, 9)

# 데이터 정보
# print(train.info()) # 결측값 없음
# print(test.info()) # 결측값 없음

# target 변수 정의
target = train.pop('TravelInsurance')

# object 변수 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # 'Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']

# object 유니크 확인
# print(train.describe(include='O')) # 2
# print(test.describe(include='O')) # 2

# 원-핫 인코딩(+ 데이터 병합)
combine = pd.concat([train, test])
combine_dummies = pd.get_dummies(combine)
n_train = len(train)
train = combine_dummies[:n_train]
test = combine_dummies[n_train:]
# print(train.shape, test.shape) # (1490, 13) (497, 13)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train,
    target,
    test_size = 0.2,
    random_state=42)

# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (1192, 13) (298, 13) (1192,) (298,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred_prob = rf.predict_proba(X_val)
# print(pred_prob[:3])

# 모델 평가
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_val, pred_prob[:, 1])
print(score) # 0.825268817204301

# 최종 예측 및 제출
pred = rf.predict_proba(test)
# print(pred[:3])

submit = pd.DataFrame({'y_pred': pred[:, 1]})
submit.to_csv('result_3.csv', index=False)

# 결과 확인
result = pd.read_csv('result_3.csv')
print(result)



0.825268817204301
     y_pred
0      0.16
1      0.13
2      0.00
3      1.00
4      0.18
..      ...
492    0.08
493    0.96
494    0.04
495    0.98
496    1.00

[497 rows x 1 columns]


# 제4회 기출

## 작업형 1유형: 문제1
'age' 컬럼의 3사분위수와 1사분위수의 차이를 절댓값으로 구하시오 (소수점 이하 버리고 정수로 출력)

In [ ]:
url_4 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch4/data4-1.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(url_4)
# print(df) # (100, 8)

age_q1 = df['age'].quantile(0.25)
age_q3 = df['age'].quantile(0.75)
# print(age_q1, age_q3) # 26.875, 77.0

# 차이(절댓값, 정수)
diff = abs(age_q1 - age_q3)
print(int(diff)) # 50

50


## 작업형 1유형: 문제2
모든 reactions 중 loves와 wows를 합한 반응 비율이 40% 보다 크고 50%보다 작은 데이터를 찾고, 이 중 type 컬럼이 'video'인 개수 구하기

In [ ]:
url_42 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch4/data4-2.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(url_42)
# print(df.head())

df['love_wow'] = df['loves'] + df['wows']
# print(df.head(1))

cond = df['love_wow'] / df['reactions']
cond_1 = cond > 0.4
cond_2 = cond < 0.5
# print(df[cond_1 & cond_2])
cond_3 = df['type'] == 'video'
# print(df[cond_1 & cond_2 & cond_3])
print(len(df[cond_1 & cond_2 & cond_3])) # 90

90


## 작업형 1유형: 문제3
'date_added' 컬럼의 날짜가 2018년 1월이면서 'country'에서 United Kingdom 단독 제작인 데이터의 개수 구하기

In [ ]:
url_43 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch4/data4-3.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(url_43)
# print(df.head(3))

# print(df.info())

# data_added 날짜 타입으로 변환
df['date_added'] = pd.to_datetime(df['date_added'])
# print(df.head(3))

df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month

# 해당 컬럼의 날짜가 2018년 1월인 데이터 찾기
cond_1 = df['year'] == 2018
cond_2 = df['month'] == 1
# print(df[cond_1 & cond_2])
# df_cond = df[cond_1 & cond_2]

# country 가 United Kingdom 인 국가
cond_3 = df['country'] == 'United Kingdom'

# print(df[cond_1 & cond_2 & cond_3])
print(len(df[cond_1 & cond_2 & cond_3])) # 6

6


## 작업형 2유형
고객의 개인 정보와 행동 패턴을 기반으로 고객의 세분화(Sgementation) 결과 예측

1. 에측 컬럼: Segmentation (세분화 결괏값; 1, 2, 3, 4)
2. 제출 파일 포함 컬럼:
- ID: 고객 식별자
- Segmentation: 예측된 세분화 결과
- 파일명: result_4.csv
3. 모델 성능은 Macro F1-SCORE 평가지표에 따라 평가

In [ ]:
train_4 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch4/train.csv"
test_4 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch4/test.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(train_4)
test = pd.read_csv(test_4)
# print(train.shape, test.shape) # (6665, 11) (2154, 10)

# print(train.head(3)) # target: Segmentation
# print(test.head(3))

# print(train.info()) # 결측치 없음
# print(test.info()) # 결측치 없음

# target 변수 설정
target = train.pop('Segmentation')

# object 변수 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # 'Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1'

# obejct 변수 unique 확인
# print(train.describe(include='O')) # 2에서 9
# print(test.describe(include='O')) # 2에서 9

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.fit_transform(test[col])
  # print(f"{col} 인코딩 완료")

# print(train.shape, test.shape) # (6665, 10) (2154, 10)
# print(train.head())

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=42)

# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (5332, 10) (1333, 10) (5332,) (1333,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 머신러닝 모델 평가
from sklearn.metrics import f1_score

score = f1_score(y_val, pred, average='macro')
print(score) # 0.5085211025257362

# 최종 예측 및 제출
pred = rf.predict(test)
# print(pred[:3])
submit = pd.DataFrame({'ID': test['ID'],
                      'Segmentation': pred})
submit.to_csv('result_4.csv', index=False)

# 결과 확인
result = pd.read_csv('result_4.csv', index_col='ID')
print(result)

0.5085211025257362
        Segmentation
ID                  
458989             2
458994             3
459000             3
459003             3
459005             1
...              ...
467950             1
467954             4
467958             1
467961             2
467968             4

[2154 rows x 1 columns]


# 제5회 기출

## 작업형 1유형: 문제1
종량제 봉투 종류가 '규격봉투'고, 종량제 봉투 용도가 '음식물쓰레기'인 2L 가격 평균을 구하라.
(가격이 0이면 평균에서 제외하고, 반올림 후 정수 출력)

In [ ]:
data_51 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch5/data5-1.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(data_51)
# print(df.shape) # (762, 26)
# print(df.head())

cond_1 = df['종량제봉투종류'] == '규격봉투'

# print(df['종량제봉투용도'].value_counts()) # 생활쓰레기, 음식물쓰레기
cond_2 = df['종량제봉투용도'] == '음식물쓰레기'

# print(df['2ℓ가격'][cond_1 & cond_2])
# print(df.isnull().sum()) # 결측치 없음
cond_3 = df['2ℓ가격'] != 0

# print(df['2ℓ가격'][cond_1 & cond_2 & cond_3])
combined_cond = df['2ℓ가격'][cond_1 & cond_2 & cond_3]
cond_mean = df['2ℓ가격'][cond_1 & cond_2 & cond_3].mean()
print(round(cond_mean)) # round 두번째 파라미터에 아무것도 넣지 않으면 정수형 반환

118


## 작업형 1유형: 문제2
bmi 수치가 정상인 사람 수와 위험 체중인 사람 수의 차이를 절댓값으로 구하기
(정수로 출력)

In [ ]:
data_52 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch5/data5-2.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(data_52)
# print(df.shape) # (10000, 3)
# print(df.head())

# bmi = weight(kg) / height ** 2
df['bmi'] = df['Weight'] / ((df['Height'] / 100) **2)
print(df.head())

# cond1 = (df['bmi'] >= 18.5) & (df['bmi'] < 23)
# cond2 = (df['bmi'] >= 23) & (df['bmi'] < 25)

# # print(df[cond1]) # 정상체중
# # print(df[cond2]) # 위험체중

# num_1 = len(df[cond1])
# num_2 = len(df[cond2])
# # print(num_1, num_2) # 1986, 2130
# result = abs(num_1 - num_2)
# print(int(result)) # 144

# apply 함수 써보기

# 저체중: bmi 18.5 미만
# 정상체중: bmi 18.5 이상 ~ 23 미만
# 과체중 또는 위험 체중: bmi 23 이상 ~ 25 미만
# 비만 체중: 25 이상

def bmi_jud(row):
  bmi = row['bmi']
  if bmi < 18.5:
    return '저체중'
  elif bmi < 23:
    return '정상체중'
  elif bmi < 25:
    return '위험체중'
  else:
    return '비만체중'

df['구분'] = df.apply(bmi_jud, axis=1)
# print(help(df.apply))
# print(df.head())

print(df['구분'].value_counts())
cond1 = (df['구분'] == '위험체중').sum()
cond2 = (df['구분'] == '정상체중').sum()

# 틀린 풀이
# cond1 = (df['구분'] == '위험체중')
# cond2 = (df['구분'] == '정상체중')
# num_cond1 = len(cond1)
# num_cond2 = len(cond2)
# print(num_cond1, num_cond2)

result = abs(cond1 - cond2)

print(int(result))

  Gender  Height  Weight        bmi
0   Male   187.6   109.7  31.170298
1   Male   174.7    73.6  24.115263
2   Male   188.2    96.5  27.245079
3   Male   182.2    99.8  30.063102
4   Male   177.5    93.6  29.708391
구분
비만체중    5842
위험체중    2130
정상체중    1986
저체중       42
Name: count, dtype: int64
144


## 작업형 1유형: 문제3
순 전입학생이 가장 많은 학교의 전체 학생 수 구하기
(순 전입학생 = 전입 학생 - 전출 학생)

In [ ]:
data_53 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch5/data5-3.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(data_53)
# print(df.shape) # (6309, 30)
# print(df.head())

# 순전입학생 구하기
df['순전입학생'] = df['전입학생수(계)'] - df['전출학생수(계)']
# print(df.head())

# 순전입학생 수 내림차순 정렬
df = df.sort_values(by='순전입학생', ascending=False)
print(df.head())

# 순전입학생이 가장 많은 학교의 전체 학생 수
# print(df['전체학생수(계)'][0]) # 기본적으로 인덱스를 재설정하지 않고 정렬된 값을 반환
# sort_values 에서 reset_index(drop=True) 필요

print(df.iloc[0, -2])

           시도교육청         지역교육청               지역   정보공시 학교코드       학교명  학교급코드  \
1880  세종특별자치시교육청    세종특별자치시교육청  세종특별자치시 세종특별자치시  S080000206    나성초등학교      2   
1382    인천광역시교육청  인천광역시서부교육지원청         인천광역시 서구  S040002035  인천한별초등학교      2   
6190     경상남도교육청       창원교육지원청   경상남도 창원시 마산합포구  S160004071  마산고운초등학교      2   
3214      경기도교육청    경기도성남교육지원청      경기도 성남시 분당구  S090007517  판교대장초등학교      2   
1407    인천광역시교육청  인천광역시동부교육지원청        인천광역시 연수구  S040002304  인천송담초등학교      2   

     설립구분 제외여부 제외사유  1학년 전입학생수  ...  5학년 전입학생수  5학년 전출학생수  5학년 전체학생수  \
1880   공립    N  NaN      159.0  ...      180.0        0.0       41.0   
1382   공립    N  NaN      179.0  ...      127.0        0.0        0.0   
6190   공립    N  NaN       37.0  ...      139.0        1.0      110.0   
3214   공립    N  NaN      150.0  ...       97.0        0.0        0.0   
1407   공립    N  NaN       10.0  ...      139.0        3.0      134.0   

      6학년 전입학생수  6학년 전출학생수  6학년 전체 학생수  전입학생수(계)  전출학생수(계)  전체학생수(계)  순전입학생  
1880    

## 작업형 2유형
문제. 차량의 특징 기반 데이터를 활용해 test 데이터의 중고차 가격 예측
1. 예측 컬럼: price (차량 가격)
2. 제출 파일명: 'result_5.csv'
3. 파일명 포함 컬럼: pred (예측된 차량 가격)
4. 모델은 RMSE(Root Mean Squared Error) 평가지표로 평가

In [ ]:
train_5 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch5/train.csv"
test_5 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch5/test.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(train_5)
test = pd.read_csv(test_5)

# print(train.shape, test.shape) # (3759, 9) (1617, 8)

# 데이터 샘플 확인
# print(train.head(3)) # price 컬럼 = target
# print(test.head(3))

# 데이터 정보 확인
# print(train.info()) # 결측치 없음
# print(test.info()) # 결측치 없음

# target 변수 설정
target = train.pop('price')

# object 변수 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # ['model', 'transmission', 'fuelType']

# object 변수 unique 확인
# print(train.describe(include='O')) # 3부터 19까지

# 레이블 인코딩
combine = pd.concat([train, test])
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
  combine[col] = le.fit_transform(combine[col])
  # print(f"{col} 인코딩 완료")

n_train = len(train)
train = combine[:n_train]
test = combine[n_train:]
# print(train.shape, test.shape) # (3759, 8) (1617, 8)
# print(train.head(3))

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=42)

# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (3007, 8) (752, 8) (3007,) (752,)

# 머신러닝 학슴 및 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import root_mean_squared_error

score = root_mean_squared_error(y_val, pred)
# print(f"RMSE: {round(score, 3)}") #  1419.293

# 최종 예측 및 제출
pred = rf.predict(test)
# print(pred[:3])
submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_5.csv', index=False)

# 데이터 확인
result = pd.read_csv('result_5.csv')
print(result)

          pred
0     15360.16
1     16173.43
2     14749.76
3     17786.06
4      6315.36
...        ...
1612  11574.43
1613  18268.82
1614   9663.89
1615  13458.33
1616   3982.38

[1617 rows x 1 columns]


# 제 6회 기출

## 작업형 1유형: 문제1
출동 시간과 도착 시간 차이가 평균적으로 가장 오래 걸린 소방서의 시간을 분으로 변환해 출력

In [ ]:
ch6_1= "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/data6-1-1.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch6_1)
# print(df.head()) # 데이터 로드 성공

# 출동 시간과 도착 시간 차이 구하기 전에 형변환
df['출동시간'] = pd.to_datetime(df['출동시간'])
df['도착시간'] = pd.to_datetime(df['도착시간'])
# print(df.info()) # 형변환 완료

# 출동 시간과 도착 시간 차이 구하기
df['소요시간'] = df['도착시간'] - df['출동시간']
# print(df.head())

# 소방서별 평균 소요시간 구하기 (분 단위)
df['분단위'] = df['소요시간'].dt.total_seconds() / 60
# print(df.head()) # 분 단위 변환 성공
# 소방서별 평균 소요시간
fire_time = df.groupby('소방서')['분단위'].mean() # 평균
# print(fire_time)
# print(fire_time.max()) # 80.65888
print(round(fire_time.max()))

81


## 작업형 1유형: 문제2
학교에서 교사 한 명당 맡은 학생 수가 가장 많은 학교를 찾고, 그 학교의 전체 교사 수를 구하시오

In [ ]:
ch6_2= "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/data6-1-2.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch6_2)
# print(df.head(10))
# print(df.shape) # (100, 8)

# 총 학생 수 구하기
s_num = df.iloc[:, 2:] # 학생 컬럼
# print(s_num)
df['학생수'] = df.iloc[:, 2:].sum(axis=1)
# print(df.head(10))
# 교사 1인당 학생 수
df['교사1인당'] = df['학생수'] / df['교사수']
# print(df.head(10))

# 학교수 파악하기
# print(df['학교명'].value_counts()) # 중복 학교 없음
# most_school = df['교사1인당'].max()
# print(most_school) # 54.57

df = df.sort_values('교사1인당', ascending=False).reset_index(drop=True)
print(df.head(10))
print(df.iloc[0, 1])

    학교명  교사수  1학년  2학년  3학년  4학년  5학년  6학년   학생수      교사1인당
0   학교8   19  151  181  161  187  174  183  1037  54.578947
1  학교69   19  189  117  157  192  144  144   943  49.631579
2  학교49   22  163  189  192  121   99  183   947  43.045455
3  학교93   23   84  199  127  143  180  157   890  38.695652
4  학교50   21  150  154  164   78   65  195   806  38.380952
5  학교11   21  166  149  196  136   54   98   799  38.047619
6  학교40   26  164  118  162  194  196  138   972  37.384615
7  학교22   23  124  149  144   82  186  136   821  35.695652
8  학교63   22   87  106  167  160  101  153   774  35.181818
9  학교54   20  113  161  124   97  127   78   700  35.000000
19


## 작업형 1유형: 문제3
연도별로 총 범죄 건수(범죄 유형별 발생 건수의 합)의 월평균 값을 구한 후 그 값이 가장 큰 연도를 찾아 해당 연도의 총 범죄 건수의 월평균 값 출력
(반올림하여 정수 출력)

In [ ]:
ch6_3= "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/data6-1-3.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch6_3)
# print(df.head(10))
# print(df.info())

# 범죄 유형별 총 범죄 건수 구하기
n_cols = df.select_dtypes(exclude='O').columns
# print(n_cols) # 범죄 유형 컬럼
df['범죄건수'] = df[n_cols].sum(axis=1, numeric_only=True)
# print(df.head(10))

# 연도별 출력
df['연도'] = df['날짜'].str[:4]
df['연도'] = pd.to_datetime(df['연도'])
df['연도'] = df['연도'].dt.year
# print(df.head(10))
# print(df.info())

# 연도 개수 확인
# print(df['연도'].value_counts()) # 2020년~ 2024년

# 연도별 범죄건수 월평균 구하기
monthly = df.groupby('연도')['범죄건수'].mean()
print(monthly)

# 가장 큰 월평균 값 (반올림 후 정수)
print(round(monthly.max()))


연도
2020    524.000000
2021    526.500000
2022    514.916667
2023    532.833333
2024    516.666667
Name: 범죄건수, dtype: float64
533


## 작엽형 2유형
문제. 건물 데이터를 기반으로 건물의 난방 부하(에너지 효율성) 예측
1. 에측 컬럼: Heat_Load(Very Low, Low, Medium, High, Very High)
2. 제출 파일 컬럼: pred
3. 제출 파일명: 'result_6.csv'
4. 제출 모델 성능은 Macro F1-Score 평가 지표에 따라 평가

In [ ]:
train_6 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/energy_train.csv"
test_6 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/energy_test.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(train_6)
test = pd.read_csv(test_6)

# 데이터 크기 확인
# print(train.shape, test.shape) # (537, 10) (231, 9)

# 데이터 샘플 확인
# print(train.head(3)) # target 변수: Heat_Load
# print(test.head(3))

# 데이터 정보 확인
# print(train.info()) # 결측치 없음
# print(test.info()) # 결측치 없음

# 데이터 기술 통계 확인
# print(train.describe())
# print(test.describe())

# 타겟 변수 설정
target = train.pop('Heat_Load')
# print(train.shape) # (537, 9)

# object 컬럼 확인
c_cols = train.select_dtypes(include='O').columns
# print(c_cols) # 'Roof', 'Height', 'Orient', 'Heat_Load'

# object 컬럼 unique 확인
# print(train.describe(include='O')) # 2부터 4까지
# print(test.describe(include='O')) # 2부터 4까지

# 레이블 인코딩
combined = pd.concat([train, test])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for c_col in c_cols:
  combined[c_col] = le.fit_transform(combined[c_col])
  # print(f"{c_col} 인코딩 완료")

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]

# print(train.shape, test.shape) # (537, 9) (231, 9)
# print(train.head(3))
# print(test.head(3))

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (429, 9) (108, 9) (429,) (108,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import accuracy_score, f1_score

acc_score = accuracy_score(y_val, pred)
f1 = f1_score(y_val, pred, average='macro')

# print(f"정확도: {acc_score}, F1-Score: {f1}")

# 최종 예측 및 제출
pred = rf.predict(test)
# print(pred[:3])
submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_6.csv', index=False)

# 결과 확인
result = pd.read_csv('result_6.csv')
print(result.shape)
print(result)



(231, 1)
          pred
0          Low
1         High
2         High
3          Low
4          Low
..         ...
226   Very Low
227     Medium
228   Very Low
229        Low
230  Very High

[231 rows x 1 columns]


## 작업형 3유형: 문제1
문제. 감기약 위약(플라시보)을 투여받은 환자의 부작용은 항암약 위약(플라시보)을 투여받은 환자와 부작용의 차이가 있는가?

- 귀무가설: 감기약(위약)의 부작용과 항암약(위약)의 부작용은 동일하다
- 대립가설: 감기약(위약)의 부작용과 항암약(위약)의 부작용은 다르다

1. 항암약을 투여받은 환자 중 '이상 없음'의 비율을 0과 1 사이로 구하시오
2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로 유의미하게 차이가 있는지 확인하려 한다. 카이제곱 검정을 사용해 검정 통계량을 구하시오
3. 위의 p-value를 구하시오

In [ ]:
# 부작용의 알려진 발생 비율
import pandas as pd

df = pd.DataFrame({
    '구분': [1, 2, 3, 4],
    '부작용': ['아픔', '조금 아픔', '속쓰림', '이상 없음'],
    '비율': ['10%', '5%', '15%', '70%']
})
# print(df)

# 항암약(위약)을 투여받은 실제 환자 그룹에서 관찰된 부작용
df_2 = pd.DataFrame({
    '항암약': [4, 4, 3, 4, 1, 4, 1, 4, 1, 4, 4, 2, 1, 4, 2, 3, 2, 4, 4, 4]
})
# print(df_2) # 1: 아픔 2: 조금아픔 3: 속쓰림 4: 이상없음


# 1. 항암약을 투여받은 환자 중 '이상 없음'의 비율을 0과 1 사이로 구하시오
pain_dict = {1: '아픔', 2: '조금아픔', 3: '속쓰림', 4: '이상없음'}
df_2 = df_2.iloc[:, 0].map(pain_dict)
# print(df_2)

# 빈도수 확인
# print(df_2.value_counts()) # 이상없음 11
num = df_2.shape[0] # 20
# print(num)
# print(f"1번: {df_2.value_counts('이상없음')}") # 0.55
non_num = df_2.iloc[:] == '이상없음'
# print(df_2[non_num].count()) # 11

non = df_2[non_num].count()
non_ratio = non / num
print(f"1번: {non_ratio}")
# print(11/20) # 0.55

# 2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로
# 유의미하게 차이가 있는지 확인하려 한다. 카이제곱 검정을 사용해 검정 통계량을 구하시오
from scipy.stats import chisquare
# print(help(chisquare))

obs = [0.1, 0.05, 0.15, 0.7] # 관찰값
exp = [0.2, 0.15, 0.1, 0.55] # 기대값

obs_count = [x * num for x in obs]
exp_count = [x * num for x in exp]

print(obs_count)
print(exp_count)

# 카이제곱(관찰빈도값, 기대빈도값)
result = chisquare(obs_count,exp_count)
# print(result)
print(f"2번: {result.statistic}")

# 3. 위의 p-value를 구하시오
print(f"3번: {result.pvalue}")

1번: 0.55
[2.0, 1.0, 3.0, 14.0]
[4.0, 3.0, 2.0, 11.0]
2번: 3.6515151515151514
3번: 0.3016362196336777


### 교재 답
관찰값과 기대값이 반드시 동일한 "카테고리 순서"로 대응되어야 한다

In [ ]:
# 부작용의 알려진 발생 비율
import pandas as pd

df = pd.DataFrame({
    '구분': [1, 2, 3, 4],
    '부작용': ['아픔', '조금 아픔', '속쓰림', '이상 없음'],
    '비율': ['10%', '5%', '15%', '70%']
})
# print(df)

# 항암약(위약)을 투여받은 실제 환자 그룹에서 관찰된 부작용
df_2 = pd.DataFrame({
    '항암약': [4, 4, 3, 4, 1, 4, 1, 4, 1, 4, 4, 2, 1, 4, 2, 3, 2, 4, 4, 4]
})
# print(df_2) # 1: 아픔 2: 조금아픔 3: 속쓰림 4: 이상없음


# 1. 항암약을 투여받은 환자 중 '이상 없음'의 비율을 0과 1 사이로 구하시오
cnt = sum(df_2['항암약'] == 4) # 이상없음 빈도 계산
ratio = cnt / len(df_2) # 이상 없음 비율 계산
print(f"1번: {ratio}")

# 또는
print(df_2['항암약'].value_counts(normalize=True))


# 2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로 유의미하게
# 차이가 있는지 확인하려 한다. 카이제곱 검정을 사용해 검정 통계량을 구하시오
from scipy.stats import chisquare

prob = [0.1, 0.05, 0.15, 0.7] # 각 기대 빈도수 계산을 위한 각 카테고리 비율
# 기대 빈도수 계산
df_num = len(df_2)
# print(df_num) # 20
exp_count = [x * df_num for x in prob]
# print(exp_count) # [2.0, 1.0, 3.0, 14.0]

# 관찰 빈도수 계산
obs_count = df_2.value_counts().sort_index().to_list()
# print(obs_count)

stat, p_value = chisquare(obs_count, exp_count)
print(stat)

# 3. 위의 p-value를 구하시오
print(p_value)

1번: 0.55
항암약
4    0.55
1    0.20
2    0.15
3    0.10
Name: proportion, dtype: float64
6.976190476190476
0.07266054733847573


## 작업형 3유형: 문제2
다중 선형 회귀 모델을 구축하고, 각 소문제에 답하기
1. 독립변수: solar(태양에너지), wind(바람 새기), O3(오존 농도)
2. 종속변수 temperature(온도)

문1. 주어진 데이터를 기반으로 다중 선형 회귀 모델을 구축하고,
독립변수 o3의 회귀 계수를 구하시오.
문2. 문1에서 적합한 모델에서 'solar'와 'o3'이 고정된 상태에서 'wind'의 세기가 증가함에 따라 'temperature'가 감소한다.
이 회귀 모델에서 'wind'의 회귀 계수에 대한 p-value 구하기
문3. 문1에서 적합한 모델에서 solar:100, wind:5, o3:30 일 때, 예측값 구하기

In [ ]:
ch6_32= "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch6/data6-3-2.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch6_32)
# print(df.head(5))

# 문1. 주어진 데이터를 기반으로 다중 선형 회귀 모델을 구축하고,
# 독립변수 o3의 회귀 계수를 구하시오.
from statsmodels.formula.api import ols

model = ols('temperature ~ solar + wind + o3', df).fit()
# print(model.summary())
# print(help(ols))

print(f"1번- o3의 회귀 계수: {model.params['o3']}")

# 문2. 문1에서 적합한 모델에서 'solar'와 'o3'이 고정된 상태에서 'wind'의 세기가 증가함에 따라 'temperature'가 감소한다.
# 이 회귀 모델에서 'wind'의 회귀 계수에 대한 p-value 구하기
print(f"2번 - wind p-value: {model.pvalues['wind']}")

# 문3. 문1에서 적합한 모델에서 solar:100, wind:5, o3:30 일 때, 예측값 구하기
new_df = pd.DataFrame({
    'solar': [100],
    'wind': [5],
    'o3': [30]
})

value = model.predict(new_df)
print(f" 3번: {value}")


1번- o3의 회귀 계수: 0.0749385437813658
2번 - wind p-value: 0.7797177202071661
 3번: 0    21.56163
dtype: float64


# 제 7회 기출

## 작업형 1유형: 문제1
결측치가 있는 행을 제거한 후 학생이 가장 많이 수강한 과목을 찾고, 해당 과목 점수를 표준화(스탠더드 스케일)한 다음 가장 큰 표준화된 값 구하기
(반올림하여 소수 셋째 자리까지 계산)

In [ ]:
ch7_11 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/student_assessment.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch7_11)
# print(df.shape) # (2565, 4)
# print(df.head())

# 결측치가 있는 행 제거
# print(df.isnull().sum()) # 결측치 확인 - score 행 결측치 있음

df = df.dropna() # 결측치 제거
# print(df.isnull().sum()) #
# print(df.shape) # (2544, 4) -> 결측치 제거 완료

# print(df.head(10))
# 학생이 가장 많이 수강한 과목 찾기
# print(df['id_assessment'].value_counts()) # 12
# print(df['id_assessment'].value_counts().idxmax()) # 12

# 해당 과목(12) 점수를 표준화하기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# print(help(StandardScaler))
cond = df['id_assessment'] == 12
# data = df[cond]['score']
# print(data)
# # standarded = scaler.fit_transform(data)
# # print(standarded)

cond_df = df[cond]
cond_df['score'] = scaler.fit_transform(cond_df[['score']])
# print(cond_df)

# 가장 큰 표준화 값 구하기
max_standard = cond_df['score'].max()
print(round(max_standard, 3)) # 반올림하여 소수 셋째자리까지 # 2.183

2.183


/tmp/ipython-input-28-2002760267.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond_df['score'] = scaler.fit_transform(cond_df[['score']])


## 작업형 1유형: 문제2
DE1 ~ DE77 컬럼 중 주가지수의 종가 'close'와 가장 상관 관계가 높은 변수를 찾아 해당 변수의 평균값 구하기 (반올림하여 소수 넷째 자리까지 계산)

In [ ]:
ch7_12 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/stock_market.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch7_12)
# print(df.head(10))
# print(df.shape) # (1000, 80)

# 종가(close)의 상관 관계 구하기
corr_df = df.corr()
# print(corr_df)
close_corr = corr_df.loc['close']
# print(close_corr)
# print(close_corr.sort_values(ascending=False))
# print(close_corr.sort_values(ascending=False).nlargest(3)) # DE19
third_corr = close_corr.sort_values(ascending=False).index[2]
print(third_corr)

# print(df.head(10))
# # cond =  == third_corr
# # print(df[cond])
# print(help(df.columns))

cond = df['DE19']
# print(cond)

# 해당 변수의 평균값 구하기
cond_mean = cond.mean()
# print(cond_mean)
print(round(cond_mean, 4)) # 소수 넷째자리까지


DE19
-0.0016


### 교재 정답

In [ ]:
import pandas as pd

df = pd.read_csv(ch7_12)

# close와의 상관 관계 (절댓값)
df_corr = df.corr()['close'].abs()
# print(df_corr)

# 상관 관계가 높은 변수명
max_corr = df_corr.loc['DE1':'DE77'].idxmax()
print(max_corr)

# 2에서 구한 변수명의 평균값
mean = df['DE14'].mean()
print(round(mean, 4))

DE14
-0.0004


## 작업형 1유형: 문제3
IQR을 이용해 이산화탄소(CO2) 이상치 수 찾기

In [ ]:
ch7_13 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/air_quality.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch7_13)
# print(df)

# 이상치 = Q1-(IQR * 1.5) 보다 작고 / Q3+(IQR * 1.5) 보다 큰 값
# print(df['CO2'].describe())
q1 = df['CO2'].quantile(.25)
q3 = df['CO2'].quantile(.75)
# print(q1, q3) # 383.8502072254157 418.7784333384595
iqr = q3 - q1
# print(iqr) # 34.92822611304376

out_1 = q1 - (iqr * 1.5)
out_2 = q3 + (iqr * 1.5)
# print(out_1, out_2) # 331.4578680558501 471.17077250802515

# 이상치 찾기
cond = (df['CO2'] < out_1) | (df['CO2'] > out_2)
# print(df[cond])
print(len(df[cond])) # 304

304


## 작업형 2유형
W-mart 매장의 판매 데이터를 기반으로 판매액 예측
1. 예측 컬럼: total(총 판매액)
2. 제출 파일 컬럼: 1개 (pred - 예측ㄷ괸 총 판매액)
3. 제출 파일명: result_7.csv
4. 모델 성능은 RMSE 평가지표에 따라 평가

In [ ]:
train_7 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/mart_train.csv"
test_7 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/mart_test.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(train_7)
test = pd.read_csv(test_7)

# 데이터 크기
# print(train.shape, test.shape) # (700, 10) (300, 9)

# 데이터 샘플
# print(train.head(3)) # 범주형 숫자 데이터 없음. target: 'total'
# print(test.head(3)) # 범주형 숫자 데이터 없음.

# 데이터 정보
# print(train.info()) # 'rating' 컬럼 제외 object, 결측치 없음
# print(test.info()) # 'rating' 컬럼 제외 object, 결측치 없음

# target 변수 설정
target = train.pop('total')

# object 컬럼 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # 'branch', 'city', 'customer_type', 'gender', 'product_line', 'payment_method', 'time_of_day', 'day_name'

# object 컬럼 unique 확인
# print(train.describe(include='O')) # 2에서 7까지

# 레이블 인코딩
combined = pd.concat([train, test])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
  combined[col] = le.fit_transform(combined[col])
  # print(f"{col} 인코딩 완료")

# print(combined.head(3))

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]
# print(train.shape, test.shape) # (700, 9) (300, 9)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (560, 9) (140, 9) (560,) (140,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import r2_score, root_mean_squared_error

r2 = r2_score(y_val, pred)
rmse = root_mean_squared_error(y_val, pred)

print(f" R2 SCORE: {r2}, RMSE: {rmse}")

# 최종 예측 및 제출
pred = rf.predict(test)
print(pred[:3])

submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_7.csv', index=False)

# 파일 확인
result = pd.read_csv('result_7.csv')
print(result.shape)
print(result)

 R2 SCORE: -0.1311536266890685, RMSE: 414286.295130828
[523412.82  490371.84  278122.005]
(300, 1)
            pred
0    523412.8200
1    490371.8400
2    278122.0050
3    487293.1875
4    406889.9100
..           ...
295  419044.3425
296  519882.9300
297  417995.2350
298  276613.4700
299  567444.9375

[300 rows x 1 columns]


### rating 변수 빼고 해보기

In [ ]:
import pandas as pd

train = pd.read_csv(train_7)
test = pd.read_csv(test_7)

# 데이터 크기
# print(train.shape, test.shape) # (700, 10) (300, 9)

# 데이터 샘플
# print(train.head(3)) # 범주형 숫자 데이터 없음. target: 'total'
# print(test.head(3)) # 범주형 숫자 데이터 없음.

# 데이터 정보
# print(train.info()) # 'rating' 컬럼 제외 object, 결측치 없음
# print(test.info()) # 'rating' 컬럼 제외 object, 결측치 없음

# target 변수 설정
target = train.pop('total')

# object 컬럼 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # 'branch', 'city', 'customer_type', 'gender', 'product_line', 'payment_method', 'time_of_day', 'day_name'

# object 컬럼 unique 확인
# print(train.describe(include='O')) # 2에서 7까지

# 레이블 인코딩
combined = pd.concat([train, test])
rating = combined.pop('rating') # rating 변수 빼기

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
  combined[col] = le.fit_transform(combined[col])
  # print(f"{col} 인코딩 완료")

# print(combined.head(3))

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]
# print(train.shape, test.shape) # (700, 9) (300, 9)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (560, 9) (140, 9) (560,) (140,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import r2_score, root_mean_squared_error

r2 = r2_score(y_val, pred)
rmse = root_mean_squared_error(y_val, pred)

print(f" R2 SCORE: {r2}, RMSE: {rmse}")

# 최종 예측 및 제출
pred = rf.predict(test)
print(pred[:3])

submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_7.csv', index=False)

# 파일 확인
result = pd.read_csv('result_7.csv')
print(result.shape)
print(result)

 R2 SCORE: -0.06742565348740248, RMSE: 402446.90206475975
[240118.0425   374574.7005   265613.709375]
(300, 1)
              pred
0    240118.042500
1    374574.700500
2    265613.709375
3    589742.685000
4    539411.670000
..             ...
295  647162.932500
296  365111.743500
297  538896.881250
298  313027.627500
299  444145.865625

[300 rows x 1 columns]


### 원-핫 인코딩으로 해보기: 제일 좋음

In [ ]:
import pandas as pd

train = pd.read_csv(train_7)
test = pd.read_csv(test_7)

# 데이터 크기
# print(train.shape, test.shape) # (700, 10) (300, 9)

# 데이터 샘플
# print(train.head(3)) # 범주형 숫자 데이터 없음. target: 'total'
# print(test.head(3)) # 범주형 숫자 데이터 없음.

# 데이터 정보
# print(train.info()) # 'rating' 컬럼 제외 object, 결측치 없음
# print(test.info()) # 'rating' 컬럼 제외 object, 결측치 없음

# target 변수 설정
target = train.pop('total')

# object 컬럼 확인
cols = train.select_dtypes(include='O').columns
# print(cols) # 'branch', 'city', 'customer_type', 'gender', 'product_line', 'payment_method', 'time_of_day', 'day_name'

# object 컬럼 unique 확인
# print(train.describe(include='O')) # 2에서 7까지

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)


# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (560, 9) (140, 9) (560,) (140,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import r2_score, root_mean_squared_error

r2 = r2_score(y_val, pred)
rmse = root_mean_squared_error(y_val, pred)

print(f" R2 SCORE: {r2}, RMSE: {rmse}")

# 최종 예측 및 제출
pred = rf.predict(test)
print(pred[:3])

submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_7.csv', index=False)

# 파일 확인
result = pd.read_csv('result_7.csv')
print(result.shape)
print(result)

(700, 30) (300, 30)
 R2 SCORE: -0.05893894161535873, RMSE: 400843.8551622919
[359791.11   694976.4675 285842.655 ]
(300, 1)
            pred
0    359791.1100
1    694976.4675
2    285842.6550
3    640079.2125
4    461416.0950
..           ...
295  323083.0575
296  537929.1225
297  486333.6975
298  373171.9950
299  567654.5700

[300 rows x 1 columns]


## 작업형 3유형: 문제1
- 조개 정보를 나타낸 데이터에서 300개 중 앞에서부터 210개는 train으로, 90개는 test 데이터로 나눈다.
- 모델을 학습할 때는 train 데이터를 사용하고, 예측할 때는 test 데이터를 사용한다.
- 모델은 다중 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0: 암컷, 1: 수컷)
* 패널티: Lasso, Lidge 등

1-1. weight를 독립변수, gender를 종속변수로 사용해 로지스틱 회귀 모델을 만들고, weight 변수가 무게가 한 단위 증가할 때 수컷일 오즈비 값 구하기 (반올림하여 소수 넷째 자리까지 계산)

1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후 이탈도(Residual Deviance) 계산 (반올림하여 소수 둘째 자리까지 계산)

1-3. 독립변수 weight만 사용해 학습한 모델에서 test 데이터의 gender를 예측하고, Error Rate(오류율)를 구하기. (반올림하여 소수 셋째 자리까지 계산)

In [ ]:
ch7_31 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/clam.csv"

In [ ]:
import pandas as pd

data = pd.read_csv(ch7_31)
# print(data)
# print(data.shape) # (300, 6)

# 데이터셋 분할 (7:3)
train = data[:210]
test = data[210:]
# print(train.shape, test.shape) # (210, 6) (90, 6)

# # 타겟 변수 설정 -> statsmodels 사용 시에는 필요 없음
# target = train.pop('gender')
test_target = test.pop('gender')

# print(train.shape, test.shape) # (210, 5) (90, 5)

# 데이터 정보 확인
# print(train.info()) # object 변수 없음
# print(test.info()) # object 변수 없음

# 데이터 기술 통계 확인
# print(train.describe()) # 범주형 숫자 데이터 없음
# print(test.describe()) # 범주형 숫자 데이터 없음

# 로지스틱 회귀 모델
from statsmodels.formula.api import logit

# model = logit('gender ~ age + length + diameter + height + weight', train).fit()
# print(model.summary())

# 1-1. weight를 독립변수, gender를 종속변수로 사용해 로지스틱 회귀 모델을 만들고,
# weight 변수가 무게가 한 단위 증가할 때 수컷일 오즈비 값 구하기 (반올림하여 소수 넷째 자리까지 계산)

model = logit('gender ~ weight', train).fit()
# print(model.summary())

import numpy as np
print(f"수컷 오즈비: {round(np.exp(model.params['weight']), 4)}") # 1.0047

# 1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는
# 로지스틱 회귀 모델을 적합시킨 후 이탈도(Residual Deviance) 계산 (반올림하여 소수 둘째 자리까지 계산)

model = logit('gender ~ age + length + diameter + height + weight', train).fit()
# print(model.summary())
# print(f"이탈도: {round(model.resid_dev, 2)}") # 오답
# print(dir(model))
# 잔차 이탈도: 로그 우도(log-likehood) 값에 -2 곱하기
print(f"이탈도: {round(model.llf * -2, 2)}")

# 1-3. 독립변수 weight만 사용해 학습한 모델에서 test 데이터의 gender를 예측하고,
# Error Rate(오류율)를 구하기. (반올림하여 소수 셋째 자리까지 계산)
model = logit('gender ~ weight', train).fit()
pred = model.predict(test) # target 값 제외
# print(pred)
# pred 값 0.5 이상 - 1 / 0.5 미만 - 0
pred = (pred > 0.5).astype(int)
# print(pred)

# 오류율: 1- 정확도
from sklearn.metrics import accuracy_score

acc_score = accuracy_score(test_target, pred)
# print(acc_score)
print(f"오류율: {round(1 - acc_score, 3)}")



Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
수컷 오즈비: 1.0047
Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
이탈도: 286.93
Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
오류율: 0.478


## 작업형 3유형: 문제2
2-1. ERP와 가장 상관 관계가 높은 값 구하기(반올림하여 소수 셋째 자리까지 계산)

2-2. CPU 컬럼이 100 미만인 것만 찾아 EPR를 종속변수, 나머지 변수들을 독립변수로 설정해 선형 회귀 모델을 만들고 적합한 결정 계수 구하기 (반올림하여 소수 셋째 자리까지 계산)

2-3. 문제2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은 값 구하기 (반올림하여 소수 셋째 자리까지 계산)

In [ ]:
ch7_32 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch7/system_cpu.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch7_32)
# print(df)

# 2-1. ERP와 가장 상관 관계가 높은 값 구하기(반올림하여 소수 셋째 자리까지 계산)
corr_df = df.corr()['ERP']
# print(corr_df)
max_corr = corr_df.nlargest(2) # feature3
print(f"상관관계 가장 높은 값: {round(max_corr.iloc[1], 3)}") # 0.882
# print(dir(df.corr))
# print(help(df.corr))


# 2-2. CPU 컬럼이 100 미만인 것만 찾아 EPR를 종속변수, 나머지 변수들을 독립변수로 설정해
# 선형 회귀 모델을 만들고 적합한 결정 계수 구하기 (반올림하여 소수 셋째 자리까지 계산)
cond = df['CPU'] < 100
# print(df[cond])
cond_df = pd.DataFrame(df[cond])
# print(cond_df.head(3))

from statsmodels.formula.api import ols

model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', cond_df).fit()
# print(model.summary())
print(f"결정 계수: {round(model.rsquared, 3)}") # 0.755

# 2-3. 문제2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은 값 구하기
# (반올림하여 소수 셋째 자리까지 계산)
# print(model.pvalues)
# print(model.pvalues.max())
print(f"가장 높은 p값: {round(model.pvalues.max(), 3)}") # 0.684


상관관계 가장 높은 값: 0.882
결정 계수: 0.755
가장 높은 p값: 0.684


# 제8회 기출

## 작업형 1유형: 문제1
1) 대륙별 맥주 소비량의 평균을 계산하고, 평균이 가장 큰 대륙 찾기

2) 1번에서 찾은 대륙에서 맥주 소비량이 5번째로 많은 국가의 맥주 소비량 구하기

In [ ]:
ch8_1 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/drinks.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch8_1)
# print(df)

# 1) 대륙(continent)별 맥주 소비량(beer_servings)의 평균을 계산하고, 평균이 가장 큰 대륙 찾기
continent_beer = df.groupby('continent')['beer_servings'].mean()
# print(contient_beer)
# 평균이 가장 큰 대륙
max_continent = continent_beer.idxmax()
# print(continent_beer.sort_values(ascending=False))
print(f"맥주 소비량 가장 많은 대륙: {max_continent}") # Europe

# 2) 1번에서 찾은 대륙에서 맥주 소비량이 5번째로 많은 국가의 맥주 소비량 구하기
# print(df.columns)
cond = df['continent'] == 'Europe'
# print(df[cond])
europe_beer = df[cond].groupby('country')['beer_servings'].sum()
# print(europe_beer)
# 맥주 소비량 많은 유럽 국가 상위 5개
eur_5 = europe_beer.nlargest(5)
# print(eur_5)
print(f"맥주소비량 5번째 유럽국가 소비량: {eur_5.iloc[4]}")

맥주 소비량 가장 많은 대륙: Europe
맥주소비량 5번째 유럽국가 소비량: 313


## 작업형 1유형: 문제2
1) '관광객 비율'이 두 번째로 높은 국가의 '사업' 방문객 수를 a라고 정의
* 방문객 합계: 관광 + 공무 + 사업 + 기타
* 관광객 비율: 관광 / 방문객 합계

2) '관광'이 두 번째로 높은 국가의 '공무' 방문객 수를 b라고 정의

3) a와 b의 합 구하기

In [ ]:
ch8_2 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/tourist.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch8_2)
# print(df)


# 1) '관광객 비율'이 두 번째로 높은 국가의 '사업' 방문객 수를 a라고 정의
# * 방문객 합계: 관광 + 공무 + 사업 + 기타
# * 관광객 비율: 관광 / 방문객 합계
df['방문객합계'] = df.iloc[:, 1:].sum(axis=1)
df['관광객비율'] = df['관광'] / df['방문객합계']
# print(df)

# 관광객 비율이 두 번째로 높은 국가의 사업 방문객 수
cond = df.sort_values('관광객비율', ascending=False)
# print(cond)
busi_2nd = df.sort_values('관광객비율', ascending=False).reset_index(drop=True).loc[1, '사업']
# print(busi_2nd) # 203

# 2) '관광'이 두 번째로 높은 국가의 '공무' 방문객 수를 b라고 정의
cond2 = df.sort_values('관광', ascending=False) # 238
# print(cond2)
work_2nd = df.sort_values('관광', ascending=False).reset_index(drop=True).loc[1, '공무']
# print(work_2nd) # 238

# 3) a와 b의 합 구하기
result = busi_2nd + work_2nd
print(result) # 441


441


## 작업형 1유형: 문제3
1) 주어진 데이터에서 'co'와 'nmhc' 컬럼을 각각 Min-Max 스케일링 하기

2) 스케일링된 'oc', 'nmhc' 컬럼의 표준편차 구하기

3) 'co' 컬럼의 표준편차에서 'nmhc' 컬럼의 표준편차를 뺀 값을 소수점 3자리로 반올림하여 구하기

In [ ]:
ch8_3 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/chem.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch8_3)
# print(df)

# 1) 주어진 데이터에서 'co'와 'nmhc' 컬럼을 각각 Min-Max 스케일링 하기
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# print(help(scaler))

df['co_scaled'] = scaler.fit_transform(df[['co']])
df['nmhc_scaled'] = scaler.fit_transform(df[['nmhc']])

# print(df)

# 2) 스케일링된 'oc', 'nmhc' 컬럼의 표준편차 구하기
co_std = df['co_scaled'].std()
nmhc_std = df['nmhc_scaled'].std()
# print(co_std, nmhc_std) # 0.2856516497116944, 0.3030617020578397

# 3) 'co' 컬럼의 표준편차에서 'nmhc' 컬럼의 표준편차를 뺀 값을 소수점 3자리로 반올림하여 구하기
diff = co_std - nmhc_std
print(round(diff, 3))


-0.017


## 작업형 2유형
통신사에서 고객에게 청구될 총 금액 예측

1. 예측할 컬럼: TotalCharges(총 청구액)
2. 제출 파일에 포함할 컬럼: pred - 예측된 총 청구액
3. 제출 파일명: result_8.csv
4. 모델의 성능은 MAE(Mean Absolute Error) 평가지표에 따라 평가

In [ ]:
train_8 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/churn_train.csv"
test_8 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/churn_test.csv"

In [ ]:
import pandas as pd

train = pd.read_csv(train_8)
test = pd.read_csv(test_8)
# print(train.shape, test.shape) # (4116, 19) (1764, 18)
# print(train.head(10)) # target: TotalCharges 컬럼
# print(test.head(3))

# 데이터 정보
# print(train.info()) # 결측치 없음,  SeniorCitizen, tenure 컬럼 외 전부 object
# print(test.info()) # 결측치 없음,  SeniorCitizen, tenure 컬럼 외 전부 object

# 데이터 기술 통계
# print(train.describe()) # SeniorCitizen 컬럼 숫자이나, 범주형으로 보여짐
# print(test.describe())

# target 변수 설정
target = train.pop('TotalCharges')

# object 컬럼 unique 값 보기
# print(train.describe(include='O')) # 2부터 4116 까지
# print(test.describe(include='O')) # 2부터 4116까지

# 원-핫 인코딩 위해 customerID 빼놓기
train_id = train.pop('customerID')
test_id = test.pop('customerID')

# 원-핫 인코딩
combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)

n_train = len(train)

train = combined_dummies[:n_train]
test = combined_dummies[n_train:]
# print(train.shape, test.shape) # (4116, 43) (1764, 43)

# 레이블 인코딩
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

# cols = combined.select_dtypes(include='O').columns
# # print(cols)
# for col in cols:
#   combined[col] = le.fit_transform(combined[col])
#   # print(f"{col} 인코딩 완료")

# # print(combined.head(3))
# n_train = len(train)
# train = combined[:n_train]
# test = combined[n_train:]
# # print(train.shape, test.shape) # (4116, 17) (1764, 17)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (3292, 43) (824, 43) (3292,) (824,)

# 머신러닝 학습 및 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import mean_absolute_error, r2_score

r2 = r2_score(y_val, pred)
mse = mean_absolute_error(y_val, pred)
# print(f"R2: {r2}, MSE: {mse}") # 원-핫 -> R2: 0.6044114660631468, MSE: 877.1742424572353
# 레이블 인코딩 -> R2: 0.6020346779073029, MSE: 881.4131559101941

# 원-핫 인코딩으로 진행
pred = rf.predict(test)
# print(pred[:3])
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result_8.csv', index=False)

# 데이터 확인
result = pd.read_csv('result_8.csv')
print(result.shape) # (1764, 1)
print(result)

(1764, 1)
           pred
0     3612.9353
1      944.0589
2     4415.9902
3      861.5555
4     1382.2678
...         ...
1759   328.5894
1760  4036.3867
1761  4342.1311
1762  1182.8199
1763  3791.6201

[1764 rows x 1 columns]


## 작업형 3유형: 문제1 (오답)

1-1. 주어진 데이터에서 로지스틱 회귀 분석을 수행해 유의확률(p-value)이 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오.
- (종속변수: Churn, 독립변수: Churn을 제외한 모든 변수)

1-2. 유의확률(p-value)dl 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀 분석을 수행하시오.
이 회귀식의 유의한 회귀 계수(상수항 포함)의 합계를 구하시오
- 반올림하여 소수 셋째 자리까지 계산

1-3. 문제1-2에서 수행한 로지스틱 회귀식에서 'DataUsage' 변수가 5만큼 증가할 때 오즈비를 구하시오.
- 반올림하여 소수 셋째 자리까지 게산

In [ ]:
ch83_1 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/churn.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch83_1)
# print(df.shape) # (1000, 11)
# print(df)

# 1-1. 주어진 데이터에서 로지스틱 회귀 분석을 수행해 유의확률(p-value)이
# 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오.
# - (종속변수: Churn, 독립변수: Churn을 제외한 모든 변수)
# print(df.info())
# print(df.describe()) # Churn, ContractRenewal, DataPlan, CustServCalls
# cols = df['Churn', 'ContractRenewal', 'DataPlan', 'CustServCalls'].columns
# print(df.value_counts())

from statsmodels.formula.api import logit
model = logit('Churn ~ AccountWeeks + ContractRenewal + DataPlan + DataUsage + CustServCalls + DayMins + DayCalls + MonthlyCharge + OverageFee', df).fit()
# print(model.summary())
cond = model.pvalues >= 0.05
# print(cond) # 7개

# 1-2. 유의확률(p-value)dl 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀 분석을 수행하시오.
# 이 회귀식의 유의한 회귀 계수(상수항 포함)의 합계를 구하시오
# - 반올림하여 소수 셋째 자리까지 계산

# DataUsage, DayMins
model = logit('Churn ~ DataUsage + DayMins', df).fit()
# print(model.summary())
# print(model.params.sum())
param_total = model.params.sum()
print(round(param_total, 3)) # -1.213

# 1-3. 문제1-2에서 수행한 로지스틱 회귀식에서 'DataUsage' 변수가 5만큼 증가할 때 오즈비를 구하시오.
# - 반올림하여 소수 셋째 자리까지 게산
import numpy as np

data_5 = np.exp(model.params['DataUsage'])
print(round(data_5, 3)) # 0.844


Optimization terminated successfully.
         Current function value: 0.393646
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6
-1.213
0.844


### 교재 정답


In [ ]:
import pandas as pd

df = pd.read_csv(ch83_1)
# print(df)

# 1-1. 주어진 데이터에서 로지스틱 회귀 분석을 수행해 유의확률(p-value)이
# 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오.
# - (종속변수: Churn, 독립변수: Churn을 제외한 모든 변수)
from statsmodels.formula.api import logit

model = logit('Churn ~ AccountWeeks + ContractRenewal + DataPlan + DataUsage + CustServCalls + DayMins + DayCalls + MonthlyCharge + OverageFee + RoamMins ', df).fit()
# print(model.summary())

over_a = sum(model.pvalues >= 0.05)
print(f"유의하지 않은 독립변수 개수: {over_a}")

# 1-2. 유의확률(p-value)dl 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀 분석을 수행하시오.
# 이 회귀식의 유의한 회귀 계수(상수항 포함)의 합계를 구하시오
# - 반올림하여 소수 셋째 자리까지 계산
model = logit('Churn ~ DataUsage + DayMins', df).fit()
# print(model.summary())
coef_total = model.params.sum()
# print(coef_total)
print(f"유의한 회귀 계수 합계: {round(coef_total, 3)}")

# 1-3. 문제1-2에서 수행한 로지스틱 회귀식에서 'DataUsage' 변수가 5만큼 증가할 때 오즈비를 구하시오.
# - 반올림하여 소수 셋째 자리까지 게산
import numpy as np

data_5 = model.params['DataUsage'] * 5
odds_5 = np.exp(data_5)
print(f"오즈비: {round(odds_5, 3)}")

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
유의하지 않은 독립변수 개수: 8
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6
유의한 회귀 계수 합계: -1.213
오즈비: 0.428


## 작업형 3유형: 문제2
다중 선형 회귀

2-1. 주어진 데이터를 이용해 종속변수(PIQ)와 독립변수(Brain, Height, Weight)로 다중 선형 회귀 분석 수행.
유의확률이 가장 작은 변수의 회귀 계수 값 구하기
- 반올림하여 소수 셋째 자리까지 계산

2-2. 문제 2-1에 적합한 모델의 결정 계수 (R-squared) 값 구하기
- 반올림하여 소수 둘째 자리까지 계산

2-3. 뇌 크기(Brain)가 90, 키(Height)가 70, 몸무게(Weight)가 150일 때 PIQ 예측.
- 반올림하여 정수로 계산

In [ ]:
ch83_2 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch8/piq.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch83_2)
# print(df)
print(df.columns)

# 2-1. 주어진 데이터를 이용해 종속변수(PIQ)와 독립변수(Brain, Height, Weight)로 다중 선형 회귀 분석 수행.
# 유의확률이 가장 작은 변수의 회귀 계수 값 구하기
# - 반올림하여 소수 셋째 자리까지 계산
from statsmodels.formula.api import ols

model = ols('PIQ ~ Brain + Height + Weight', df).fit()
print(model.summary())
smallest_p = model.pvalues.min()
# print(smallest_p)
p_name = model.pvalues.idxmin()
print(p_name)
# print(f"유의확률 가장 적은 변수 회귀계수: {round(smallest_coef, 3)}") # -2.644 - > 틀렸음
print(f"유의확률 가장 적은 변수 회귀계수: {round(model.params[p_name], 3)}") # 2.343


# # 2-2. 문제 2-1에 적합한 모델의 결정 계수 (R-squared) 값 구하기
# # - 반올림하여 소수 둘째 자리까지 계산
# rsq = model.rsquared
# print(f"모델의 결정 계수: {round(rsq, 2)}") # 0.37

# # 2-3. 뇌 크기(Brain)가 90, 키(Height)가 70, 몸무게(Weight)가 150일 때 PIQ 예측.
# # - 반올림하여 정수로 계산
# new_df = pd.DataFrame({'Brain': [90],
#                        'Height': [70],
#                        'Weight': [150]})
# pred = model.predict(new_df)
# print(pred)
# print(f"예측 PIQ: {int(round(pred))}") # 106


Index(['PIQ', 'Brain', 'Height', 'Weight'], dtype='object')
                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Thu, 19 Jun 2025   Prob (F-statistic):           8.25e-05
Time:                        15:09:42   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

# 제9회 기출

## 작업형 1유형: 문제1
1) 총대출액은 신용대출과 담보대출의 합이다

2) 각 지역코드 내에서, 성별(예: 1과2)별로 '총대출액'의 합계 계산

3) 성별 간 총대출액의 차이가 가장 큰 지역코드 구하기

In [ ]:
ch_91 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/loan.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch_91)
# print(df)

# 1) 총대출액은 신용대출과 담보대출의 합이다
df['총대출액'] = df['신용대출'] + df['담보대출']
# print(df)

# 2) 각 지역코드 내에서, 성별(예: 1과2)별로 '총대출액'의 합계 계산
gender_loan = df.groupby(['지역코드', '성별'])['총대출액'].sum()
# print(gender_loan)
# print(gender_loan.unstack())
# print(gender_df)
# print(help(pd.crosstab))

# 3) 성별 간 총대출액의 차이가 가장 큰 지역코드 구하기
gender_loan = gender_loan.unstack()
# print(gender_loan)
gender_loan['diff'] = abs(gender_loan[1] - gender_loan[2]) # 차이를 구하는 것이므로 abs 사용
# print(gender_loan)
# print(gender_loan.sort_values('diff', ascending=False)) # 4100000278
max_diff = gender_loan['diff'].idxmax()
print(max_diff)

4100000278


## 작업형 1유형: 문제2
1) 검거율 = 검거건수 / 발생건수

2) 각 연도에서 가장 높은 검거율을 기록한 범죄유형 찾기

3) 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산
- 정수로 출력

### 교재 답안1

In [ ]:
ch_92 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/crime.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(ch_92)
df

,연도,구분,강력범죄,절도범죄,폭력범죄,지능범죄,풍속범죄,특별경제범죄,마약범죄,보건범죄,환경범죄,교통범죄,노동범죄,선거범죄,병역범죄,기타범죄
0,2014,발생건수,800,900,750,650,700,1034,600,850,800,900,750,800,700,1000
1,2014,검거건수,500,600,400,450,400,961,350,500,450,550,400,450,400,600
2,2015,발생건수,900,768,800,750,700,1000,850,800,900,950,800,850,750,1000
3,2015,검거건수,600,713,500,550,400,600,500,450,550,600,450,500,400,600
4,2016,발생건수,850,800,880,750,700,950,900,800,850,900,750,800,700,950
5,2016,검거건수,500,600,812,550,400,600,500,450,500,550,400,450,400,600
6,2017,발생건수,1000,900,850,800,750,950,1400,900,1000,950,800,850,750,900
7,2017,검거건수,600,500,500,550,400,600,1300,500,550,600,450,500,400,500
8,2018,발생건수,1450,900,850,800,750,950,900,800,850,900,750,800,700,950
9,2018,검거건수,1350,600,500,550,400,600,500,450,500,550,400,450,400,600


In [ ]:
import pandas as pd

df = pd.read_csv(ch_92)
# print(df)

# 1) 검거율 = 검거건수 / 발생건수
cond_1 = df['구분'] == '발생건수'
cond_2 = df['구분'] == '검거건수'
df1 = df[cond_1].iloc[:, 2:]
df2 = df[cond_2].iloc[:, 2:]
# print(df1)
# print(df2)

# 검거율 계산
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
# print(df1)
# print(df2)
df3 = df2 / df1
# print(df3)

# 2) 각 연도에서 가장 높은 검거율을 기록한 범죄유형 찾기
listbox = df3.idxmax(axis=1)
# print(listbox)

# 3) 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산
# - 정수로 출력
result = 0
for index, item in enumerate(listbox):
  result = result + df2.loc[index, item]

print(int(result))

7799


### 교재 답안2

In [ ]:
# melt 사용
df_long = pd.melt(df, id_vars=['연도', '구분'], var_name='범죄유형', value_name='건수')
# print(df_long)

# 피벗 사용
df_pivot = df_long.pivot_table(index=['연도', '범죄유형'], columns='구분', values='건수')
# print(df_pivot)
df_pivot['검거율'] = df_pivot['검거건수'] / df_pivot['발생건수']
# print(df_pivot)

# 각 연도에서 가장 높은 검거율을 기록한 범죄유형 찾기
listbox = df_pivot.groupby(['연도'])['검거율'].idxmax()
# print(listbox)
max_crime = df_pivot.loc[listbox]
# print(max_crime)

# 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산
total_crime = max_crime['검거건수'].sum()
print(int(total_crime))


7799


In [ ]:
import pandas as pd

df = pd.read_csv(ch_92)
# print(df)

# df = df.stack()
# print(df)
# print(df.index)
df = df.set_index(['연도', '구분']).stack()
# print(df)
df = df.unstack(level=1)
# print(df)

# # 검거율 구하기
df['검거율'] = df['검거건수'] / df['발생건수']
# print(df)


# 각 연도에서 가장 높은 검거율을 기록한 범죄유형 찾기
yearly_df = df.groupby('연도')['검거율'].idxmax()
# print(yearly_df)

# 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산
total = df.loc[yearly_df]['검거건수'].sum()
print(total)

7799


### 좀 더 쉽게 하기

In [ ]:
import pandas as pd

df = pd.read_csv(ch_92)

df = df.set_index(['연도', '구분']).stack().unstack(level=1)
df['검거율'] = df['검거건수'] / df['발생건수']

# 각 연도에서 가장 높은 검거율 기록한 범죄유형 찾기
yearly_df = df.groupby('연도')['검거율'].idxmax()

# 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산
result = df.loc[yearly_df,'검거건수'].sum()
print(result)

7799


### 연습하기

1. DataFrame을 **'범죄종류'**와 '구분' 컬럼으로 분리하세요.

- 예: '강도_발생'을 '강도'와 '발생'으로 분리
- '범죄종류'와 '구분' 컬럼을 생성하고 기존 '범죄종류_구분' 컬럼은 삭제해도 좋습니다.

2. 각 범죄종류에 대해 검거율을 계산하세요. (검거율 = 검거 / 발생)

3. 각 연도별, 월별로 가장 높은 검거율을 기록한 범죄종류를 찾으세요.

4. 가장 높은 검거율을 기록한 범죄종류의 검거건수를 모두 합산하여 정수로 출력하세요.


In [ ]:
import pandas as pd
import numpy as np

data = {
    '연도': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
    '월': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    '강도_발생': [10, 12, 8, 15, 11, 9, 13, 10, 7],
    '강도_검거': [8, 10, 7, 13, 9, 8, 11, 8, 6],
    '절도_발생': [20, 25, 18, 22, 28, 20, 25, 23, 19],
    '절도_검거': [15, 20, 16, 18, 23, 17, 20, 19, 15],
    '살인_발생': [2, 3, 1, 3, 2, 1, 2, 1, 0],
    '살인_검거': [2, 3, 1, 3, 2, 1, 2, 1, 0]
}
df = pd.DataFrame(data)

print("원본 DataFrame:")
print(df)

원본 DataFrame:
     연도  월  강도_발생  강도_검거  절도_발생  절도_검거  살인_발생  살인_검거
0  2020  1     10      8     20     15      2      2
1  2020  2     12     10     25     20      3      3
2  2020  3      8      7     18     16      1      1
3  2021  1     15     13     22     18      3      3
4  2021  2     11      9     28     23      2      2
5  2021  3      9      8     20     17      1      1
6  2022  1     13     11     25     20      2      2
7  2022  2     10      8     23     19      1      1
8  2022  3      7      6     19     15      0      0


In [ ]:
data = {
    '연도': [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022],
    '강도_발생': [10, 12, 8, 15, 11, 9, 13, 10, 7],
    '강도_검거': [8, 10, 7, 13, 9, 8, 11, 8, 6],
    '절도_발생': [20, 25, 18, 22, 28, 20, 25, 23, 19],
    '절도_검거': [15, 20, 16, 18, 23, 17, 20, 19, 15],
    '살인_발생': [2, 3, 1, 3, 2, 1, 2, 1, 0],
    '살인_검거': [2, 3, 1, 3, 2, 1, 2, 1, 0]
}
df = pd.DataFrame(data)

df = df.set_index('연도').stack()
df = df.reset_index() # 멀티 인덱스 시리즈를 데이터프레임으로 변환하고 인덱스를 컬럼으로 복원
df.columns = ['연도', '범죄종류_구분', '건수']

# 1. DataFrame을 **'범죄종류'**와 '구분' 컬럼으로 분리하세요.
# - 예: '강도_발생'을 '강도'와 '발생'으로 분리
# - '범죄종류'와 '구분' 컬럼을 생성하고 기존 '범죄종류_구분' 컬럼은 삭제해도 좋습니다.
df[['범죄종류', '구분']] = df['범죄종류_구분'].str.split('_', expand=True)
# print(df)

# 2. 각 범죄종류에 대해 검거율을 계산하세요. (검거율 = 검거 / 발생)
df = df.drop(columns = '범죄종류_구분')
df = df.groupby(['연도', '범죄종류','구분'])['건수'].sum()
df = df.unstack(level=2)

df['검거율'] = df['검거'] / df['발생']
# print(df)

# 3. 각 연도별로 가장 높은 검거율을 기록한 범죄종류를 찾으세요.
year_max = df.groupby('연도')['검거율'].idxmax()
# print(year_max)

# 4. 가장 높은 검거율을 기록한 범죄종류의 검거건수를 모두 합산하여 정수로 출력하세요.
total = df.loc[year_max]['검거'].sum()
print(total)

15


## 작업형 1유형: 문제3
1) 만족도의 결측치는 만족도의 평균값으로 대체

2) 근속연수의 결측치는 같은 부서 내에서 동일한 성과등급을 가진 직원들의 평균 근속연수로 대체. 이때, 평균 근속연수는 소수점 이하를 절사하여 정수로 변환하여 사용

3) 각 직원의 '연봉/근속연수' 값을 계산하여, 그 값이 세 번째로 큰 직원의 근속연수 구하기.

4) 각 직원의 '연봉/만족도' 값을 계산하여, 그 값이 두 번째로 큰 직원의 교육 참가 횟수 구하기

5) 3과 4의 값을 더한 결과 구하기 (정수 출력)

In [265]:
ch_93 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/hr.csv"

In [310]:
import pandas as pd

df = pd.read_csv(ch_93)
# print(df)

# 1) 만족도의 결측치는 만족도의 평균값으로 대체
satis_mean = df['만족도'].mean()
# print(satis_mean) # 5.017045454545454
# print(df['만족도'].isnull().sum()) # 120 행
df['만족도'] = df['만족도'].fillna(satis_mean)
# print(df['만족도'].isnull().sum()) # 결측치 채우기 완료
# print(df.info())

# 2) 근속연수의 결측치는 같은 부서 내에서 동일한 성과등급을 가진 직원들의 평균 근속연수로 대체.
# 이때, 평균 근속연수는 소수점 이하를 절사하여 정수로 변환하여 사용
# print(df['근속연수'].isnull().sum()) # 48건
part_group = df.groupby(['부서', '성과등급'])['근속연수'].transform('mean').astype(int)
# print(part_group)

df['근속연수'] = df['근속연수'].fillna(part_group)
# print(df)
# print(df['근속연수'].isnull().sum()) # 결측치 채움

# 3) 각 직원의 '연봉/근속연수' 값을 계산하여, 그 값이 세 번째로 큰 직원의 근속연수 구하기.
df['연근'] = df['연봉'] / df['근속연수']
# print(df)
third = df['연근'].nlargest(3).index[2]
# print(third) # 801
third_y = df.loc[third]['근속연수']
print(third_y) # 1
# print(df.loc[801])


# 4) 각 직원의 '연봉/만족도' 값을 계산하여, 그 값이 두 번째로 큰 직원의 교육 참가 횟수 구하기
df['연만'] = df['연봉'] / df['만족도']
# print(df)
sec = df['연만'].nlargest(2).index[1]
# print(sec) # 737
# print(df.sort_values('연만', ascending=False))
sec_edu = df.loc[sec, '교육참가횟수']
print(sec_edu) # 6
# print(df.loc[737])

# 5) 3과 4의 값을 더한 결과 구하기 (정수 출력)
result = third_y + sec_edu
print(int(result))


1.0
6
7


## 작업형 2유형
문제. 농작물에서 농약 검출 여부 예측

1. 예측할 컬럼: 농약검출여부 (0: 미검출, 1: 검출, 2:재검사필요)
2. 제출 파일의 포함될 컬럼: pred - 예측된 검출 여부
3. 제출 파일명: result_9.csv
4. 모델의 성능은 macro F1 score 평가지표에 따라 평가

In [311]:
train_9 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/farm_train.csv"
test_9 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/farm_test.csv"

In [323]:
import pandas as pd

train = pd.read_csv(train_9)
test = pd.read_csv(test_9)
# print(train.shape, test.shape) # (4000, 9) (1000, 8)
# print(train.head(3)) # 농약검출여부 : target
# print(test.head(3))

# 타겟 변수 설정
target = train.pop('농약검출여부')
# print(target.value_counts()) # 2: 1989, 0: 1758, 1:253

# 데이터 정보 확인
# print(train.info()) # 결측치 없음
# print(test.info()) # 결측치 없음

# 데이터 기술 통계 확인
# print(train.describe())
# print(test.describe())

# object 컬럼 unique 확인
# print(train.describe(include='O')) # 3부터 15까지
# print(test.describe(include='O')) # 3부터 15까지

# 레이블 인코딩 위한 데이터 병합
combined = pd.concat([train, test])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cols = combined.select_dtypes(include='O').columns
for col in cols:
  combined[col] = le.fit_transform(combined[col])
  # print(f"{col} 인코딩 완료")

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]
# print(train.shape, test.shape) # (4000, 8) (1000, 8)
# print(train.head(3)) # 인코딩 완료
# print(test.head(3)) # 인코딩 완료

# 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape) # (3200, 8) (800, 8) (3200,) (800,)

# 머신러닝 학습 및 예측 (분류)
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred[:3])

# 모델 평가
from sklearn.metrics import accuracy_score, f1_score

acc = accuracy_score(y_val, pred)
f1 = f1_score(y_val, pred, average='macro')
# print(f"정확도: {acc}, F1: {f1}") # 정확도: 0.9625, F1: 0.8680387463168264

# 최종 예측 및 제출
pred = rf.predict(test)
# print(pred[:3])
submit = pd.DataFrame({'pred': pred})
submit.to_csv('result_9.csv', index=False)

# 파일 확인
result = pd.read_csv('result_9.csv')
print(result.shape)
print(result)

(1000, 1)
     pred
0       2
1       0
2       0
3       2
4       0
..    ...
995     2
996     2
997     2
998     2
999     0

[1000 rows x 1 columns]


## 작업형 3유형: 문제1 (다중선형회귀)

1-1. train 데이터에서 c1, c2, c3, c4, c5를 독립변수로, design을 종속변수로 하는 다중 회귀모형을 적합한 후, 절편을 제외한 독립변수 중 p-value가 0.05보다 작은 변수의 개수 구하기

1-2. 1-1에서 적합한 회귀모형의 결과를 바탕으로, p-value가 0.05 이하인 유의한 독립변수만을 선택하여 새로운 다중 회귀모형을 적합한 후, train 데이터에서 design의 예측값을 산출하고 예측값과 실제 design 값 사이의 피어슨 상관계수를 반올림하여 소수 셋째 자리까지 계산

1-3. 1-2에서 적합한 회귀모형을 이용하여 test 데이터에서 design의 예측값을 산출한 후, 예측값과 실제 design값 사이의 RMSE(Root Mean Squared Error) 를 반올림하여 소수 셋째 자리까지 계산


In [324]:
ch_931 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/design.csv"

In [360]:
import pandas as pd

df = pd.read_csv(ch_931)
# print(df)

# print(df.shape) # (200, 7)
train = df.iloc[:140]
test = df.iloc[140:]
# print(train.shape, test.shape) # (140, 7) (60, 7)

# 1-1. train 데이터에서 c1, c2, c3, c4, c5를 독립변수로, design을 종속변수로 하는 다중 회귀모형을 적합한 후, 절편을 제외한 독립변수 중 p-value가 0.05보다 작은 변수의 개수 구하기
from statsmodels.formula.api import ols

model = ols('design ~ c1 + c2 + c3 + c4 + c5', train).fit()
# print(model.summary())
cond1 = model.pvalues[1:] < 0.05
# print(cond1) # 3개 -> c1, c2, c4
num_cond1 = sum(cond1)
print(f"1번: {num_cond1}")

# 1-2. 1-1에서 적합한 회귀모형의 결과를 바탕으로, p-value가 0.05 이하인 유의한 독립변수만을 선택하여 새로운 다중 회귀모형을 적합한 후,
#  train 데이터에서 design의 예측값을 산출하고 예측값과 실제 design 값 사이의 피어슨 상관계수를 반올림하여 소수 셋째 자리까지 계산
model = ols('design ~ c1 + c2 + c4', train).fit()
# print(model.summary())
# pred = model.predict(train)
# print(pred)

train['pred'] = model.predict(train)
train['resi'] = train['design'] - train['pred']
# print(train)

corr = train['design'].corr(train['pred'])
# print(corr)
# print(help(df.corr))
print(f"2번: {round(corr, 3)}")

# 1-3. 1-2에서 적합한 회귀모형을 이용하여 test 데이터에서 design의 예측값을 산출한 후,
# 예측값과 실제 design값 사이의 RMSE(Root Mean Squared Error) 를 반올림하여 소수 셋째 자리까지 계산
test['pred'] = model.predict(test)

# ==== 오답 =====
# test['resi'] = test['design'] - test['pred']
# # print(test)
# import numpy as np

# mean_error = test['resi'].mean() # -2.0415499325921296 -> 에러 평균
# # print(mean_error)
# squared_merr = mean_error **2
# # print(squared_merr) # 4.167926127266929 -> 제곱 에러 평균
# rmse = np.sqrt(squared_merr)
# # print(rmse) # 2.0415499325921296
# print(f"3번: {round(rmse, 3)}") # 3번: 2.042


# ===정답====
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(test['design'], test['pred'])
# print(rmse)
print(f"3번: {round(rmse, 3)}") # 8.488

1번: 3
2번: 0.501
3번: 8.488


/tmp/ipython-input-360-2961406375.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred'] = model.predict(train)
/tmp/ipython-input-360-2961406375.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['resi'] = train['design'] - train['pred']
/tmp/ipython-input-360-2961406375.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

## 작업형 3유형: 문제2 (로지스틱 회귀)

- 종속변수: Churn (고객 이탈 여부 - 1: 이탈, 0: 유지)
- 독립변수: MonthlyCharges(월 사용 요금), CustomerTenure(고객 유지 기간), HasPhoneService(전화 서비스 이용 여부), HasTechInsurance(기술 보험 가입 여부)

2-1. 주어진 데이터를 사용하여 고객 이탈(Churn)을 예측하는 로지스틱 회귀 모형을 적합한 후, MonthlyCharges 변수의 p-value를 반올림하여 소수 셋째 자리까지 구하기

2-2. 적합한 로지스틱 회귀모형을 바탕으로, 전화 서비스를 받지 않은 고객이 이탈할 확률 대비 전화 서비스를 받은 고객이 이탈할 확률에 대한 오즈비(odds ratio) 계산
- 반올림하여 소수 셋째 자리까지 계산

2-3. 적합한 로지스틱 회귀모형을 이용하여 각 고객의 이탈 확률을 예측한 후, 예측된 이탈 확률이 0.3을 초과하는 고객의 수 구하기

In [361]:
ch_932 = "https://github.com/lovedlim/bigdata_analyst_cert_v2/raw/refs/heads/main/part4/ch9/retention.csv"

In [372]:
import pandas as pd

df = pd.read_csv(ch_932)
# print(df)

from statsmodels.formula.api import logit
# 2-1. 주어진 데이터를 사용하여 고객 이탈(Churn)을 예측하는 로지스틱 회귀 모형을 적합한 후,
# MonthlyCharges 변수의 p-value를 반올림하여 소수 셋째 자리까지 구하기
model = logit('Churn ~ + MonthlyCharges + CustomerTenure + HasPhoneService + HasTechInsurance', df).fit()
# print(model.summary()) # 로지스틱 회귀 모형 적합
print(round(model.pvalues['MonthlyCharges'], 3)) # 0.008

# 2-2. 적합한 로지스틱 회귀모형을 바탕으로, 전화 서비스를 받지 않은 고객이 이탈할 확률 대비
# 전화 서비스를 받은 고객이 이탈할 확률에 대한 오즈비(odds ratio) 계산
# - 반올림하여 소수 셋째 자리까지 계산
import numpy as np

odds = np.exp(model.params['HasPhoneService'])
print(round(odds, 3)) # 0.701


# 2-3. 적합한 로지스틱 회귀모형을 이용하여 각 고객의 이탈 확률을 예측한 후, 예측된 이탈 확률이 0.3을 초과하는 고객의 수 구하기
pred = model.predict(df)
# print(pred)
cond = pred > 0.3
print(sum(cond)) # 65


Optimization terminated successfully.
         Current function value: 0.582234
         Iterations 6
0.008
0.701
65
